In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import scipy
import urllib.request
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [ ]:
train = ImageDataGenerator(rescale = 1./255.)

test = ImageDataGenerator(rescale = 1./255.)

train_dataset = train.flow_from_directory("data/Train",
                                          target_size=(150,150),
                                          batch_size = 10,
                                          class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("data/Test",
                                      target_size=(150,150),
                                      batch_size =10,
                                      class_mode = 'binary')

In [ ]:
inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
urllib.request.urlretrieve(
    inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape=(150,150,3),include_top=False,weights=None)
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable=False

In [ ]:
last_layer =  pre_trained_model.get_layer('mixed7')
last_ouput = last_layer.output
x = keras.layers.Flatten()(last_ouput)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(train_dataset,
    epochs=15,
    validation_data=test_dataset,
)

In [ ]:
def predictImage(filename):
    img1 = image.load_img(filename,target_size=(150,150))
    
    plt.imshow(img1)
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    value_predicted = model.predict(X)
    print(value_predicted)
    if value_predicted == 0:
        plt.xlabel("Anemia",fontsize=30)
    else:
        plt.xlabel("NonAnemia",fontsize=30)

In [ ]:
# predictImage(r"")

In [ ]:
model.summary()

In [ ]:
# model.save("model.h5")